In [1]:
pip install -U langgraph

     ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
     ------------------------------------ --- 41.0/44.4 kB 2.0 MB/s eta 0:00:01
     -------------------------------------- 44.4/44.4 kB 437.4 kB/s eta 0:00:00
     ---------------------------------------- 0.0/42.9 kB ? eta -:--:--
     ---------------------------------------- 42.9/42.9 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/145.0 kB ? eta -:--:--
   ---------------------------------------  143.4/145.0 kB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 145.0/145.0 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/423.3 kB ? eta -:--:--
   ----------- ---------------------------- 122.9/423.3 kB 7.5 MB/s eta 0:00:01
   --------------------------------- ------ 358.4/423.3 kB 5.6 MB/s eta 0:00:01
   ---------------------------------------  419.8/423.3 kB 5.3 MB/s eta 0:00:01
   ---------------------------------------- 423.3/423.3 kB 3.3 MB/s eta 0:


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Setup

In [3]:
%pip install -U langgraph langsmith langchain_anthropic


     ---------------------------------------- 0.0/65.2 kB ? eta -:--:--
     ---------------------------------------- 65.2/65.2 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/243.4 kB ? eta -:--:--
   ------------------------------ --------- 184.3/243.4 kB 3.8 MB/s eta 0:00:01
   ---------------------------------------- 243.4/243.4 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/443.6 kB ? eta -:--:--
   --------------- ------------------------ 174.1/443.6 kB 5.3 MB/s eta 0:00:01
   ---------------------------------------  440.3/443.6 kB 6.8 MB/s eta 0:00:01
   ---------------------------------------- 443.6/443.6 kB 4.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.0 MB 10.7 MB/s eta 0:00:01
   --------------------- ------------------ 1.1/2.0 MB 11.2 MB/s eta 0:00:01
   ------------------------ --------------- 1.2/2.0 MB 9.4 MB/s eta 0:00:01
   ---


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")

Part 1: Build a Basic Chatbot¶
We'll first create a simple chatbot using LangGraph. This chatbot will respond directly to user messages. Though simple, it will illustrate the core concepts of building with LangGraph. By the end of this section, you will have a built rudimentary chatbot.

Start by creating a StateGraph. A StateGraph object defines the structure of our chatbot as a "state machine". We'll add nodes to represent the llm and functions our chatbot can call and edges to specify how the bot should transition between these functions.

In [5]:
from typing import Annotated
from typing_extensions import TypedDict

# Importing necessary components from the LangGraph library
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


# Define a state structure using TypedDict
class State(TypedDict):
    # The state has a key called "messages" which is a list.
    # The `add_messages` function in the annotation specifies how updates to this key should be handled.
    # Instead of overwriting the list, `add_messages` ensures that new messages are appended to the existing list.
    messages: Annotated[list, add_messages]


# Create a graph builder using the StateGraph class.
# This graph will use the `State` structure to define its nodes (states).
graph_builder = StateGraph(State)

# At this point, the graph builder is ready to define and manage states and their transitions.
# The `StateGraph` can be used to create workflows, processes, or any system that requires state management.

Our graph can now handle two key tasks:

Each node can receive the current State as input and output an update to the state.
Updates to messages will be appended to the existing list rather than overwriting it, thanks to the prebuilt add_messages function used with the Annotated syntax.

When defining a graph, the first step is to define its State. The State includes the graph's schema and reducer functions that handle state updates. In our example, State is a TypedDict with one key: messages. The add_messages reducer function is used to append new messages to the list instead of overwriting it. Keys without a reducer annotation will overwrite previous values.

Next, add a "chatbot" node. Nodes represent units of work. They are typically regular python functions.

In [6]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")


def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)

the chatbot node function takes the current State as input and returns a dictionary containing an updated messages list under the key "messages". This is the basic pattern for all LangGraph node functions.

The add_messages function in our State will append the llm's response messages to whatever messages are already in the state.

Next, add an entry point. This tells our graph where to start its work each time we run it.

In [ ]:
graph_builder.add_edge(START, "chatbot")